In [119]:
#------------------------------------------------------------------------------------------
# IMPORT MODULES
#------------------------------------------------------------------------------------------
import datetime
import numpy as np
import pandas as pd
from geopy.distance import geodesic
import functools
import time
from pyproj import Geod

import plotly
import plotly.graph_objs as go
import plotly.io as pio
from IPython.display import display, HTML
import plotly.offline as py

from io import BytesIO
from zipfile import ZipFile
from urllib.request import urlopen

from ipywidgets import interactive, HBox, VBox, IntSlider, Label, Dropdown, Button, Output



#------------------------------------------------------------------------------------------
# PROCESS AIRPORT DATASET
#------------------------------------------------------------------------------------------
# Get  the airport dataset
r = urlopen('https://www.partow.net/downloads/GlobalAirportDatabase.zip')
z = ZipFile(BytesIO(r.read()))

for name in z.namelist():
    if name == 'GlobalAirportDatabase.txt':
        file = z.open(name)
        df = pd.read_csv(file, sep=':', header=None)
        
# Rename columns and delete useless ones
df.columns = ["ICAO Code",
              "IATA Code",
              "Airport Name",
              "City/Town",
              "Country",
              "Latitude Degrees",
              "Latitude Minutes",
              "Latitude Seconds",
              "Latitude Direction",
              "Longitude Degrees",
              "Longitude Minutes",
              "Longitude Seconds",
              "Longitude Direction",
              "Altitude",
              "Latitude Decimal Degrees",
              "Longitude Decimal Degrees"]
df.drop(columns=["ICAO Code",
              "IATA Code",
              "Airport Name",
              "Latitude Degrees",
              "Latitude Minutes",
              "Latitude Seconds",
              "Latitude Direction",
              "Longitude Degrees",
              "Longitude Minutes",
              "Longitude Seconds",
              "Longitude Direction",
              "Altitude"], inplace=True)

# Make sure that every string is in capital letter
df['City/Town'] = df['City/Town'].str.upper() 
df['Country'] = df['Country'].str.upper() 


# Drop duplicates by City/Town
df = df.drop_duplicates(subset='City/Town', keep='first')
df.reset_index(drop=True, inplace=True)

# Get lat et lon of Helsinki
lat = df.loc[df['City/Town'] == 'HELSINKI', 'Latitude Decimal Degrees'].iloc[0]
lon = df.loc[df['City/Town'] == 'HELSINKI', 'Longitude Decimal Degrees'].iloc[0]
origin = (lat, lon)

# Compute distances from Helsinki to *
def compute_dist(row, origin):
    """
    Compute distance between Helsinki and given cooridnate.
    """
    lat = row['Latitude Decimal Degrees']
    lon = row['Longitude Decimal Degrees']
    destination = (lat, lon)
    return geodesic(origin, destination).kilometers

df['Distance'] = df.apply(lambda row: compute_dist(row, origin), axis=1)


#------------------------------------------------------------------------------------------
# CREATE INITIAL FIGURE
#------------------------------------------------------------------------------------------
# Drop-down list to select the type of meat
dropdown = Dropdown(
    options=['Beef','Lamb','Pork','Chicken'],
    value='Beef',
    description='Meat: ',
)

# Slider to select the frequency of consumption
slider = IntSlider(
    value=1,
    min=1,
    max=7,
    step=1,
    description='Frequency: ',
    continuous_update=False
)

# Button to play animation
button = Button(description='Play animation')
out = Output()

# Put all together
container = HBox(children=[dropdown, slider, button, out])


# Map plot
trace = go.Scattergeo()
layout = go.Layout(
    showlegend = False,
    #template="plotly_dark",
    title=dict(text=''),
    barmode='overlay',
    geo = dict(
        showland = True,
        showcountries = True,
        showocean = True,
        countrywidth = 0.5,
        landcolor = 'rgb(230, 145, 56)',
        lakecolor = 'rgb(0, 255, 255)',
        oceancolor = 'rgb(0, 255, 255)',
        projection = dict(
            type = 'orthographic',
            rotation = dict(
                lon = -100,
                lat = 40,
                roll = 0
            )
        ),
        lonaxis = dict(
            showgrid = True,
            gridcolor = 'rgb(102, 102, 102)',
            gridwidth = 0.5
        ),
        lataxis = dict(
            showgrid = True,
            gridcolor = 'rgb(102, 102, 102)',
            gridwidth = 0.5
        )
    )
)
g = go.FigureWidget(data=[trace], layout=layout)


#------------------------------------------------------------------------------------------
# IMPLEMENT CALLBACK FUNCTION FOR SLIDER AND DROPDOWN
#------------------------------------------------------------------------------------------
# Kg of proteins in 1kg of meat (source: https://www.nutrition.org.uk/nutritionscience/nutrients-food-and-ingredients/protein.html?start=4&__cf_chl_jschl_tk__=ca2354198a312edc211325e646025ab4b6311b6f-1575136256-0-AY2DS4_oQnTBV_P2vrTsfXXZvXMEtt-XSkHA5vPhMUaU7T4GOn7pFCIB2kFXEmJB0E-fW7VnvrDIoMwcmjmNNoTppwhHaY4JEELJkWTsXCT72toDEzIL92AYkorbkBGvb6-R0EWs2SEo_jCDfUdZQ8iAR_7kvj3FaRR8JllNm1UU1lshF5JF76Vw5yxgwqJwVMHvbDOth28gFFYlf9QXSUcgw7rGczIE5WXd2WLtbe8-3FCAylkYpgCsu3x1XmHMlSfyjW2JsApP1HNJmVIIC-0bdbmO7XVj-dX6tT5H6YygdUFU09tiWXkcOSUJSvWTdUWiPwp0EXqjQcln6ZpGDyGOFs05ymfDOS7QD5jgiOchYkK8CUmtw2x7vDJ5m8jRHf2h4yLkIh5lpdQ-44qVL5dBJ2B7x1MJJ37-cVjpMK5U)
chicken_prot_1kg = 0.32
beef_prot_1kg = 0.31
lamb_prot_1kg = 0.292
pork_prot_1kg = 0.316

# Kg of CO2-eq for 100g of proteins (source: https://ora.ox.ac.uk/objects/uuid:b0b53649-5e93-4415-bf07-6b0b1227172f/download_file?file_format=pdf&safe_filename=Reducing_foods_environment_impacts_Science%2B360%2B6392%2B987%2B-%2BAccepted%2BManuscript.pdf&type_of_work=Journal+article)
beef_CO2 = 50
lamb_CO2 = 20
pork_CO2 = 7.6
chicken_CO2 = 5.7

# Hypothesis 1: One serving of meat per person is 0.2kg
serving = 0.2
# ypothesis 2: CO2 emissions from a Boeing 737-400 per passenger: 115g per passenger km (source: https://www.carbonindependent.org/22.html)
passenger_co2 = 0.115

# Init variables stroing randomly chosen airport
dest_city = ''
dest_lat = 0.0
dest_lon = 0.0
dest_dist = 0.0

# Coordinates of Helsinki
HEL_lat = df.loc[df['City/Town'] == 'HELSINKI', 'Latitude Decimal Degrees'].iloc[0]
HEL_lon = df.loc[df['City/Town'] == 'HELSINKI', 'Longitude Decimal Degrees'].iloc[0]


# Callback function for slider and dropdowns
def response(change):
    """
    Response function due to the changes in dropdown or slider.
    """
    # Global variables
    global dest_city, dest_lat, dest_lon, dest_dist
    global HEL_lat, HEL_lon

    # Get the valeus of the dropdown menus
    meat = dropdown.value
    frequency = slider.value
    
    # Make the computations for CO2 equivalence per year depending on the meat
    if meat == 'Beef':
        co2eq = (beef_CO2*beef_prot_1kg*serving) * frequency * 52
        
    if meat == 'Lamb':
        co2eq = (lamb_CO2*lamb_prot_1kg*serving) * frequency * 52
            
    if meat == 'Pork':
        co2eq = (pork_CO2*pork_prot_1kg*serving) * frequency * 52
            
    if meat == 'Chicken':
        co2eq = (chicken_CO2*chicken_prot_1kg*serving) * frequency * 52
        
        
    # Compute distance for that amount of CO2
    eq_dist = co2eq/passenger_co2
    
    # Find an airport for that distance
    radius = 50
    tmp_df = df[df['Distance'].abs().between(eq_dist-radius, eq_dist+radius)]
    rows = np.random.choice(tmp_df.index.values, 1)
    sampled_df = tmp_df.loc[rows]
    
    # Get info about the randomly selected airport
    dest_city = sampled_df.iloc[0]['City/Town']
    dest_lat = sampled_df.iloc[0]['Latitude Decimal Degrees']
    dest_lon = sampled_df.iloc[0]['Longitude Decimal Degrees']
    dest_dist = sampled_df.iloc[0]['Distance']

    # Update figure
    with g.batch_update():                
        # Create temporary dataset
        Data = {'Latitude': [HEL_lat, dest_lat],
                'Longitude':[HEL_lon, dest_lon],
                'City/Town':['HELSINKI', dest_city]}
        line_df = pd.DataFrame (Data, columns = ['Latitude','Longitude', 'City/Town'])
        
        # Update figure
        g.data[0].lon = line_df['Longitude']
        g.data[0].lat = line_df['Latitude']
        g.data[0].text = line_df['City/Town']
        g.data[0].mode = 'lines'
        g.data[0].line = dict(width = 2, color = 'rgb(213,62,79)')
        
        # Update title
        g.layout.title = 'From HELSINKI to '+dest_city+ ' ('+str(int(dest_dist))+' km)'
        
        
#------------------------------------------------------------------------------------------
# IMPLEMENT CALLBACK FUNCTION FOR ANIMATION BUTTON
#------------------------------------------------------------------------------------------
def on_button_clicked(b, N):
    """
    """
    # Global variables
    global dest_city, dest_lat, dest_lon, dest_dist
    global HEL_lat, HEL_lon
    
    # Compute the extra points between the two coordinates
    geoid = Geod(ellps="WGS84")
    extra_points = geoid.npts(HEL_lon, HEL_lat, dest_lon, dest_lat, N)
    extraPoints_df = pd.DataFrame(extra_points, columns=['lon', 'lat'])
    
    # Add coordinates of origin (Helsinki) and destination in the dataframe
    top_row = pd.DataFrame({'lon':[HEL_lon],'lat':[HEL_lat]})
    last_row = pd.DataFrame({'lon':[dest_lon],'lat':[dest_lat]})
    extraPoints_df = pd.concat([top_row, extraPoints_df, last_row]).reset_index(drop=True)
    
    # Play animation with a loop
    for k in range(extraPoints_df.shape[0]):
        with g.batch_update():
            g.data[0].lon = pd.DataFrame([extraPoints_df.iloc[k]['lon']], columns = ['lon'])['lon']
            g.data[0].lat = pd.DataFrame([extraPoints_df.iloc[k]['lat']], columns = ['lat'])['lat']
            g.data[0].mode = 'markers'
            g.data[0].marker = dict(color="red", size=10)
        time.sleep(1)
        

#------------------------------------------------------------------------------------------
# LISTEN TO THE INTERACTIVE WIDGETS
#------------------------------------------------------------------------------------------
dropdown.observe(response, names='value')
slider.observe(response, names='value')
button.on_click(functools.partial(on_button_clicked, N=10))


#------------------------------------------------------------------------------------------
# DISPLAY FIGURE
#------------------------------------------------------------------------------------------
vb = VBox([container, g])
vb.layout.align_items = 'center'
vb

In [29]:
Data = {'Latitude': [20, 60],
        'Longitude':[-10, 30],
        'City/Town':['CITY_1', 'CITY_2']}
line_df = pd.DataFrame (Data, columns = ['Latitude','Longitude', 'City/Town'])

# Interpolate between






trace = go.Scattergeo(
        lon = line_df['Longitude'],
        lat = line_df['Latitude'],
        text = line_df['City/Town'],
        mode = 'lines',
        line = dict(width = 2, color = 'rgb(213,62,79)')
)

layout = go.Layout(
    showlegend = False,
    #template="plotly_dark",
    title=dict(text='My Title'),
    barmode='overlay',
    geo = dict(
        showland = True,
        showcountries = True,
        showocean = True,
        countrywidth = 0.5,
        landcolor = 'rgb(230, 145, 56)',
        lakecolor = 'rgb(0, 255, 255)',
        oceancolor = 'rgb(0, 255, 255)',
        projection = dict(
            type = 'orthographic',
            rotation = dict(
                lon = -100,
                lat = 40,
                roll = 0
            )
        ),
        lonaxis = dict(
            showgrid = True,
            gridcolor = 'rgb(102, 102, 102)',
            gridwidth = 0.5
        ),
        lataxis = dict(
            showgrid = True,
            gridcolor = 'rgb(102, 102, 102)',
            gridwidth = 0.5
        )
    ),
    updatemenus=[dict(type="buttons",
                          buttons=[dict(label="Play",
                                        method="animate",
                                        args=[None])])]
)

frames=[go.Frame(
        data=[go.Scattergeo(
            lon=pd.DataFrame([extraPoints_df.iloc[k]['lon']], columns = ['lon'])['lon'],
            lat=pd.DataFrame([extraPoints_df.iloc[k]['lat']], columns = ['lat'])['lat'],
            mode="markers",
            marker=dict(color="blue", size=10)
            )]
        )
        for k in range(N)]


g = go.Figure(data=trace, layout=layout, frames=frames)
fig = go.FigureWidget(g)

ValueError: 
Frames are not supported by the plotly.graph_objs.FigureWidget class.
Note: Frames are supported by the plotly.graph_objs.Figure class

In [49]:


def on_button_clicked(b, params="some_default_string"):
    with out:
        print(params)
        
button = Button(description="Button")
out = Output()
button.on_click(functools.partial(on_button_clicked, params="abcdefg"))
VBox([button,out])

In [65]:
from ipywidgets.embed import embed_minimal_html, dependency_state

s1 = IntSlider(max=200, value=100)
s2 = IntSlider(value=40)
containertest = HBox(children=[s1, s2])
embed_minimal_html('export.html', views=[containertest], state=dependency_state([s1, s2]))

In [112]:
print(dest_lon)
print(dest_lat)

-110.36200000000001
24.072


In [111]:
print(HEL_lon)
print(HEL_lat)

25.043000000000003
60.254


In [117]:
geoid = Geod(ellps="WGS84")
extra_points = geoid.npts(HEL_lon, HEL_lat, dest_lon, dest_lat, 10)
extraPoints_df = pd.DataFrame(extra_points, columns=['lon', 'lat'])

# Add rows
top_row = pd.DataFrame({'lon':[HEL_lon],'lat':[HEL_lat]})
last_row = pd.DataFrame({'lon':[dest_lon],'lat':[dest_lat]})
extraPoints_df = pd.concat([top_row, extraPoints_df, last_row]).reset_index(drop=True)
extraPoints_df

,lon,lat
0,25.043000,60.254000
1,12.428179,65.905122
2,-6.032830,69.998521
3,-30.036798,71.475070
4,-53.747154,69.755066
5,-71.696264,65.501682
6,-83.927546,59.763444
7,-92.371269,53.226332
8,-98.513662,46.244707
9,-103.245469,38.999611


In [118]:
extraPoints_df.shape[0]

12